In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

In [22]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN, self).__init__()
    # N_out = floor( (N_in+2p-k)/2 ) + 1,
    # Where, N_in = number of input features
    #        N_out = number of output features
    #        k = convolution kernel size
    #        s = convolution stride size

    self.conv1 = nn.Conv2d(
        in_channels=in_channels,
        out_channels=8,
        kernel_size=(3, 3), # same convolution
        stride=(1, 1),
        padding=(1, 1),
    ) # 28 features
    self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # 14 features
    self.conv2 = nn.Conv2d(
        in_channels=8,
        out_channels=16,
        kernel_size=(3, 3), # same convolution
        stride=(1, 1),
        padding=(1, 1),
    )
    self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # 7 features
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x

In [23]:
model = CNN()
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)

torch.Size([64, 10])


In [24]:
def save_checkpoint(state, filename='my_checkpoint.pth.tar'):
  print('=> Saving checkpoint')
  torch.save(state, filename)

def load_checkpoint(checkpoint):
  print('=> Loading checkpoint')
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10
load_model = False

In [27]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)

In [28]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: dataset/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [29]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
train_loader

In [30]:
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: dataset/
    Split: Test
    StandardTransform
Transform: ToTensor()

In [31]:
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
test_loader

In [32]:
model = CNN().to(device)

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [34]:
if load_model:
  load_checkpoint(torch.load("my_checkpoint.pth.tar"))

In [35]:
for epoch in tqdm(range(num_epochs)):
  losses = []

  if epoch % 2 == 0:
    checkpoint = {
        'optimizer': optimizer.state_dict(),
        'state_dict': model.state_dict(),
    }
    save_checkpoint(checkpoint)

  for batch_idx, (data, targets) in enumerate(train_loader):
    # Get the data to cuda if possible
    data = data.to(device=device) # 64, 1, 28, 28
    targets = targets.to(device=device)

    # Forward pass
    scores = model(data)

    # Calculate the loss
    loss = criterion(scores, targets)

    # Backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()
  print(f'Loss at epoch {epoch} is {loss.item()}')


  0%|          | 0/10 [00:00<?, ?it/s]

=> Saving checkpoint


 10%|█         | 1/10 [00:18<02:44, 18.23s/it]

Loss at epoch 0 is 0.014146321453154087


 20%|██        | 2/10 [00:36<02:27, 18.45s/it]

...

Loss at epoch 7 is 0.14296755194664001
=> Saving checkpoint


 90%|█████████ | 9/10 [02:42<00:17, 17.96s/it]

Loss at epoch 8 is 0.0054262252524495125


100%|██████████| 10/10 [03:00<00:00, 18.01s/it]

Loss at epoch 9 is 0.02800786681473255


In [36]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(dim=1)

      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f"Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100}")

  model.train()

In [37]:
check_accuracy(train_loader, model)

Checking accuracy on training data
Got 59579/60000 with accuracy 99.29833333333333


In [38]:
check_accuracy(test_loader, model)

Checking accuracy on test data
Got 9872/10000 with accuracy 98.72


In [39]:
load_checkpoint(torch.load("my_checkpoint.pth.tar"))

=> Loading checkpoint


<ipython-input-39-c59bbf0f433d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_checkpoint(torch.load("my_checkpoint.pth.tar"))


In [40]:
ll

total 124
drwxr-xr-x 3 root   4096 Feb 28 11:02 dataset/
-rw-r--r-- 1 root 116582 Feb 28 11:16 my_checkpoint.pth.tar
drwxr-xr-x 1 root   4096 Feb 26 18:27 sample_data/
